In [1]:
#import libraries
import boto3
import codecs
import csv
import json
import time

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
# file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
# folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

##### Next cell is to be run only if there is the need to correct an error concerning the file extension of some documents that have wrongly assigned as txt when they were pdf

In [ ]:
# prefix = "english_documents/raw_pdf/"
# i = 0
# for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
#     srcKey = object.key
#     if ".txt" in srcKey:
#         print(i)
#         newKey = srcKey.replace(".txt", ".pdf")
#         copySource = s3BucketName + '/' + srcKey         
#         s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
#         s3.Object(s3BucketName, srcKey).delete()
#     i += 1

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [4]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

##### To get the list of documents that haven't been transformed yet

In [24]:
i = 0

file_key = "metadata/India_metadata.csv"
obj_key_dict = {}
obj_key_dict_pdf = {}

obj = s3.Object(bucket_name = s3BucketName, key = file_key)
for row in csv.reader(codecs.getreader("utf-8")(obj.get()['Body'])):
#         print(row[3])
    obj_key_dict["english_documents/text_files/" + row[3][:-4]] = 0
    obj_key_dict_pdf["english_documents/raw_pdf/" + row[3][:-4]] = 0
    i += 1
print(f"There are {i} elements in the database out of which {len(obj_key_dict)} are unique")

j = 0
k = 0
Ti = time.perf_counter()
in_dict = {}
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix='english_documents/text_files/'):
#     print(obj.key[:-4])
    if obj.key[:-4] in  obj_key_dict:
        in_dict[obj.key[:-4]] = 0
#         print(obj.key)
        j += 1
out_dict = {}
for item in obj_key_dict:
    if item not in in_dict:
        item = item.replace("text_files", "raw_pdf")
        out_dict[item] = 0
        k += 1
        
Tf = time.perf_counter()
print(f"There are {j} objects from the S3 bucket in the database. \n There are {k} files in the database which haven't been transformed\n The omputation has taken {Tf - Ti:0.4f} seconds")

There are 1040 elements in the database out of which 1040 are unique
There are 394 objects from the S3 bucket in the database. 
 There are 646 files in the database which haven't been transformed
 The omputation has taken 8.9110 seconds


In [25]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "english_documents/raw_pdf/"

i = 0

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key:# and i > 48: #lower_limit < i < upper_limit:
        if obj.key[:-4] in obj_key_dict_pdf:# out_dict:
#             print(i, "**", obj.key)
            i += 1
#             key = obj.key.replace("raw_pdf", "text_files").replace(".pdf", ".txt")
#             policy = AWS_pdf_to_text(obj.key, s3BucketName)
#             if policy != "":
#                 s3.Object(s3BucketName, key).put(Body = policy)
    #         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
print(f"There are {i} documents to be transformed")  

There are 400 documents to be transformed


In [21]:
len(out_dict)

646